In [1]:
import joblib
EXP_PATH_NAME="WACA-IF"
joblib.cpu_count()

32

In [2]:
# !pip install --upgrade pip

import os
from joblib import Parallel, delayed

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import dataclasses
from sklearn.svm import OneClassSVM
from dataclasses import asdict
from tqdm import tqdm
import warnings
import random
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold # Feature selector

pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
import seaborn as sns


# Global utitlity functions are in separate notebook
%run ./Classification_utility-functions.ipynb
%run ./SEED-CONSTANTS.ipynb
RIVAL_CLASSIFIER_TYPE_LST=["OCSVM", "kNN", "LOF"]


np.random.seed(SEED)
print(f"Numpy Seed was set to: {SEED}")

print("Setup Complete")

SEED: 567
FINAL_EXP_RESULTS_PATH: clip=False_experiments_results
--------------------SEED and CONSTANTS imported--------------------
SEED: 567
Python 3.9.10
EER: 0.333, Threshold: 0.600 <-- Arbitrary case
EER: 0.000, Threshold: 0.900 <-- Best case
EER: 1.000, Threshold: 0.900 <-- Worse case
EER: 0.400, Threshold: 0.200 <-- Worse case
EER: 0.167, Threshold: 0.600 <-- Arbitrary case
EER: 0.000, Threshold: 0.900 <-- Best case
EER: 1.000, Threshold: 0.900 <-- Worse case
EER: 0.333, Threshold: 1.000 <-- Worse case
--------------------Utility functions imported--------------------
SEED: 567
FINAL_EXP_RESULTS_PATH: clip=False_experiments_results
--------------------SEED and CONSTANTS imported--------------------
SEED: 567
Python 3.9.10
--------------------Preprocessing utility functions imported--------------------
SEED: 567
FINAL_EXP_RESULTS_PATH: clip=False_experiments_results
--------------------SEED and CONSTANTS imported--------------------
SEED: 567
Python 3.9.10
--------------------Neu

In [3]:
import sklearn
sklearn.__dir__()
sklearn.__version__

'1.0.2'

In [4]:
@dataclasses.dataclass
class ExperimentParameters:
    """Contains all relevant parameters to run an experiment."""

    name: str  # Name of Parameter set. Used as identifier for charts etc.
    frequency: int
    max_subjects: int
    max_test_subjects: int
        
    user_ids: list
    num_sample_points_per_exp: int
    exp_begin_cutoff_idx: int
    exp_end_cutoff_idx: int
        
    
    seconds_per_subject_train: float
    seconds_per_subject_test: float
    window_size: int  # After resampling
    ocsvm_step_width: int  # After resampling
    scaler: str  # StandardScaler, MinMaxScaler, Normalizer, MaxAbsScaler, RobustScaler, PowerTransformer
    scaler_scope: str  # {"subject", "session"}
    scaler_global: bool  # fit transform scale on all data (True) or fit on training only (False)
    ocsvm_kernel: str # ocsvm kernel
    ocsvm_nu: float  # Best value found in random search, used for final model
    ocsvm_gamma: float  # Best value found in random search, used for final model
    feature_cols: list  # Columns used as features
    exclude_subjects: list  # Don't load data from those users
        
    # Calculated values
    def __post_init__(self):
        # HDF key of table:
        self.table_name = f"sensors_{self.frequency}hz"

        

# INSTANCES
# ===========================================================

# NAIVE_APPROACH
# -----------------------------------------------------------
NAIVE_MINMAX_OCSVM = ExperimentParameters(
    name="NAIVE-MINMAX_OCSVM",
    frequency=100,
    max_subjects=29,
    max_test_subjects=10,
    user_ids = [1, 2, 3, 4, 5, 6, 7, 8, 19, 21, 22, 26, 27, 28, 29, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 49],
    num_sample_points_per_exp=21000,
    exp_begin_cutoff_idx=500,
    exp_end_cutoff_idx=-500,
    seconds_per_subject_train=210,
    seconds_per_subject_test=210,    
    window_size=250,
    ocsvm_step_width=250,
    scaler="minmax",
    scaler_scope="subject",
    scaler_global=True,
    ocsvm_kernel="rbf",
    ocsvm_nu=None,
    ocsvm_gamma=None,
    feature_cols=[
        "x_a",
        "y_a",
        "z_a",
        "x_g",
        "y_g",
        "z_g",
    ],
    exclude_subjects=[],
)

# VALID_APPROACH
# -----------------------------------------------------------
VALID_MINMAX_OCSVM = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-MINMAX-OCSVM",
    scaler_global=False,
    ocsvm_nu=0.165,
    ocsvm_gamma=0.039,
)

# NAIVE_ROBUST_APPROACH
# -----------------------------------------------------------
NAIVE_ROBUST_OCSVM = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="NAIVE-ROBUST-OCSVM",
    scaler="robust",
    scaler_global=True,
    ocsvm_nu=0.153,
    ocsvm_gamma=0.091,  # below median, selected by chart
)

# ROBUST_APPROACH (VALID)
# -----------------------------------------------------------
VALID_ROBUST_OCSVM_125 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=125
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)


VALID_ROBUST_OCSVM_250 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=250
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_500 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=500
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_750 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=750
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_1000 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=1000
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_1250 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=1250
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_1500 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=1500
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_1750 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=1750
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_2000 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=2000
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

# NORMALIZER_APPROACH (VALID)
# -----------------------------------------------------------
VALID_NORMALIZER_OCSVM = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-NORMALIZER-OCSVM",
    scaler="Normalizer",
    scaler_global=False,
    ocsvm_nu=0.074,
    ocsvm_gamma= 0.029,
)

In [5]:
P = VALID_ROBUST_OCSVM_2000
P.ocsvm_step_width = int(P.window_size * .5)
P.classifier="IF"

P.train_cores=1 # 20 cores for every user and 1 core for the nested crossval function
P.test_cores=2 # 10 cores for every user and 2 for the nested crossval function

P.scaler_clip=False

# param_dist
param_dist = {
    "model__n_estimators": np.linspace(50, 200, num=10, dtype=np.int64),
    'model__max_samples':  np.arange(0.1, 1.1, 0.1),
    'model__contamination': np.arange(0.01, 0.11, 0.01),
    'model__max_features': np.arange(0.1, 1.1, 0.1),
    # 'model__bootstrap': [True, False], 
    }

param_dist = {
    "model__n_estimators": [10],
    'model__max_samples':  np.arange(0.1, 1.1, 0.1),
    'model__contamination': np.arange(0.01, 0.11, 0.01),
    'model__max_features': np.arange(0.1, 1.1, 0.1),
    # 'model__bootstrap': [True, False], 
    }

param_dist = {
    "model__n_estimators": [50, 100, 200],
    'model__max_samples':  np.arange(0.1, 1.01, 0.1),
    'model__contamination': np.linspace(0.0001, 0.1, 50),
    'model__max_features': np.arange(0.1, 1.01, 0.1),
    'model__warm_start': [True], 
    }

# param_dist = {
#     "model__n_estimators": [50, 100, 150, 200], #np.linspace(50, 200, num=10, dtype=np.int64),
#     'model__max_samples':  [.5,  .1],
#     'model__contamination': [0.01],
#     'model__max_features': [.5, 1.0],
#     'model__warm_start': [True], 
#     }

# param_dist = {
#     "model__n_estimators": [50, 100], #np.linspace(50, 200, num=10, dtype=np.int64),
#     'model__max_samples':  [.1],
#     'model__contamination': [0.01],
#     'model__max_features': [.5, 1.0],
#     'model__warm_start': [True], 
#     }



In [6]:
param_dist

{'model__n_estimators': [50, 100, 200],
 'model__max_samples': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]),
 'model__contamination': array([0.0001    , 0.00213878, 0.00417755, 0.00621633, 0.0082551 ,
        0.01029388, 0.01233265, 0.01437143, 0.0164102 , 0.01844898,
        0.02048776, 0.02252653, 0.02456531, 0.02660408, 0.02864286,
        0.03068163, 0.03272041, 0.03475918, 0.03679796, 0.03883673,
        0.04087551, 0.04291429, 0.04495306, 0.04699184, 0.04903061,
        0.05106939, 0.05310816, 0.05514694, 0.05718571, 0.05922449,
        0.06126327, 0.06330204, 0.06534082, 0.06737959, 0.06941837,
        0.07145714, 0.07349592, 0.07553469, 0.07757347, 0.07961224,
        0.08165102, 0.0836898 , 0.08572857, 0.08776735, 0.08980612,
        0.0918449 , 0.09388367, 0.09592245, 0.09796122, 0.1       ]),
 'model__max_features': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]),
 'model__warm_start': [True]}

In [7]:
utils_ppp(P)

,Value
name,VALID-ROBUST-OCSVM
frequency,100
max_subjects,29
max_test_subjects,10
user_ids,"[1, 2, 3, 4, 5, 6, 7, 8, 19, 21, 22, 26, 27, 2..."
num_sample_points_per_exp,21000
exp_begin_cutoff_idx,500
exp_end_cutoff_idx,-500
seconds_per_subject_train,210
seconds_per_subject_test,210


In [8]:
utils_eer_scorer

make_scorer(utils_eer, greater_is_better=False)

In [9]:
#include 47 later
# user_ids = [9]
df_exps_dict = load_data_frames(P.user_ids, P.exp_begin_cutoff_idx, P.exp_end_cutoff_idx, P.num_sample_points_per_exp)
raw_dfList_exp1, raw_dfList_exp2 = df_exps_dict['dfList_exp1'], df_exps_dict['dfList_exp2']

Loading exp1 data:
1) accel_count: 28388, gyro_count: 31997
2) accel_count: 26010, gyro_count: 28954
3) accel_count: 28227, gyro_count: 31814
4) accel_count: 24860, gyro_count: 26105
5) accel_count: 24270, gyro_count: 24347
6) accel_count: 25012, gyro_count: 25060
7) accel_count: 25301, gyro_count: 25382
8) accel_count: 21975, gyro_count: 21658
19) accel_count: 24110, gyro_count: 25050
21) accel_count: 24326, gyro_count: 23809
22) accel_count: 29123, gyro_count: 28724
26) accel_count: 23148, gyro_count: 24291
27) accel_count: 24299, gyro_count: 23589
28) accel_count: 23807, gyro_count: 24523
29) accel_count: 24030, gyro_count: 23457
35) accel_count: 24388, gyro_count: 23673
36) accel_count: 24228, gyro_count: 24208
37) accel_count: 31945, gyro_count: 31816
38) accel_count: 22135, gyro_count: 22327
39) accel_count: 23573, gyro_count: 23459
40) accel_count: 23057, gyro_count: 24296
41) accel_count: 24102, gyro_count: 23681
42) accel_count: 24074, gyro_count: 24328
43) accel_count: 22631,

In [10]:
randomized_data_idx = list(range(len(P.user_ids)))
random.Random(SEED).shuffle(randomized_data_idx)
split_idx = 2 * (len(randomized_data_idx)//3) + 1
train_set = randomized_data_idx[: split_idx]
test_set = randomized_data_idx[split_idx: ]
# train_set = randomized_data_idx
print(f"train_set: {train_set}\ntest_set: {test_set}")
# train_set = test_set
# test_set = train_set
print(f"train_set: {train_set}\ntest_set: {test_set}")

train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]


In [11]:
num_sample_points_per_exp_user_47 = 18000
df_exps_dict_user_47 = load_data_frames([47], P.exp_begin_cutoff_idx, P.exp_end_cutoff_idx, num_sample_points_per_exp_user_47)
dfList_exp1_user_47, dfList_exp2_user_47 = df_exps_dict_user_47['dfList_exp1'], df_exps_dict_user_47['dfList_exp2']

raw_dfList_exp1_user_47 = dfList_exp1_user_47
raw_dfList_exp2_user_47 = dfList_exp2_user_47

Loading exp1 data:
47) accel_count: 22777, gyro_count: 22226
Loading exp2 data:
47) accel_count: 17718, gyro_count: 18353


In [12]:
print(f"train_set: {train_set}")
# print(f"X_exp1_train_dic: {X_exp1_train_dic.keys()}")
# print(f"X_exp2_train_dic: {X_exp2_train_dic.keys()}")

train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5]


In [13]:
print(f"test_set: {test_set}")
# print(f"X_exp1_test_dic: {X_exp1_test_dic.keys()}")
# print(f"X_exp2_test_dic: {X_exp2_test_dic.keys()}")

test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]


# 0. No Smoothing
### Optimizing and Testing

In [15]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = None



preprocessing_method=None
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/clip={P.scaler_clip}-Smoothing={P.smoothing}-Prep={preprocessing_method}-EER_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/clip={P.scaler_clip}-Smoothing={P.smoothing}-Prep={preprocessing_method}-EER_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{train_file_name}" + "-"*40 + "\n")
    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")
    



EER_df_train_dict={}
EER_df_test_dict={}


dfList_dict={
            "dfList_exp1": raw_dfList_exp1,
            "dfList_exp2": raw_dfList_exp2,
            "dfList_exp1_user_47": raw_dfList_exp1_user_47,
            "dfList_exp2_user_47": raw_dfList_exp2_user_47
}

test_dict_key=DASH_MACRO_NUM
EER_df_train_dict[test_dict_key] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                                extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                                param_dist=param_dist)

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\ntest_dict_key: {test_dict_key}\n")
    f.write(EER_df_train_dict[test_dict_key].to_string())

        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

reseting experiment params successful!
train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]


  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 337
len_exp2_user_47: 289
MakeWACAXExpDicOwner Time:  32.76443340443075
MakeWACAXExpDicUnknown Time:  74.40389953926206
Done extracting features



  0%|          | 0/3 [00:00<?, ?it/s]

run: 0: 100%|██████████| 20/20 [00:00<00:00, 15098.29it/s]

 33%|███▎      | 1/3 [07:50<15:41, 470.55s/it]

run: 1: 100%|██████████| 20/20 [00:00<00:00, 19553.86it/s]

 67%|██████▋   | 2/3 [15:10<07:32, 452.77s/it]

run: 2: 100%|██████████| 20/20 [00:00<00:00, 14383.76it/s]

100%|██████████| 3/3 [23:03<00:00, 461.29s/it]

  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 20/20 [00:00<00:00, 16794.01it/s]

 20%|██        | 1/5 [00:09<00:39,  9.82s/it]

run: 1: 100%|██████████| 20/20 [00:00<00:00, 12164.45it/s]

 40%|████      | 2/5 [00:18<00:27,  9.24s/it]

run: 2: 100%|██████████| 20/20 [00:00<00:00, 18682.87it/s]

 60%|██████    | 3/5 [00:27<00:18,  9.22s/it]

run: 3: 100%|██████████| 20/20 [00:00<00:00, 14979.66it/s]

 80%|████████  | 4/5 [00:36<00:09,  9.16s/it]

run: 4: 100%|██████████| 20/20 [00:00<00:00, 16939.84it/s]

 11%|█         | 1/9 [25:41<3:25:34, 1541.86s/it]

window_size: 250
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 167
len_exp2_user_47: 143
MakeWACAXExpDicOwner Time:  7.022921136580408
MakeWACAXExpDicUnknown Time:  36.148808342404664
Done extracting features


...


  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 10610.43it/s]

 20%|██        | 1/5 [00:03<00:13,  3.35s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 6731.35it/s]

 40%|████      | 2/5 [00:06<00:10,  3.37s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 6668.21it/s]

 60%|██████    | 3/5 [00:10<00:06,  3.34s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 7823.73it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.36s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 8466.50it/s]

 78%|███████▊  | 7/9 [03:34<00:50, 25.45s/it]

window_size: 1750
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  0.921557248570025
MakeWACAXExpDicUnknown Time:  3.4645930230617523
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 10951.19it/s]

 20%|██        | 1/5 [00:03<00:13,  3.34s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 11208.72it/s]

 40%|████      | 2/5 [00:06<00:10,  3.37s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 4950.78it/s]

 60%|██████    | 3/5 [00:10<00:06,  3.36s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 10680.68it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.37s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 9366.47it/s]

 89%|████████▉ | 8/9 [03:55<00:24, 24.13s/it]

window_size: 2000
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  0.8299801731482148
MakeWACAXExpDicUnknown Time:  3.1670140521600842
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 9379.03it/s]

 20%|██        | 1/5 [00:03<00:13,  3.31s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 6064.64it/s]

 40%|████      | 2/5 [00:06<00:09,  3.27s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 7788.87it/s]

 60%|██████    | 3/5 [00:09<00:06,  3.28s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 5786.84it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.27s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 10697.03it/s]

100%|██████████| 9/9 [04:16<00:00, 28.46s/it]


# 1. Butterworth frequency Cut-off

## 1.1 Naive Approach
### Optimizing and Testing

In [14]:
init_experiment_params(exp_config=P)

print(f"test_set: {test_set}")
P.smoothing = "Butterworth"

preprocessing_method="Naive"
time_of_execution = time.strftime("%Y-%m-%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/clip={P.scaler_clip}-Smoothing={P.smoothing}-Prep={preprocessing_method}-EER_df_test_dict.txt"
old_test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/old_{time_of_execution}_{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"


EER_df_test_dict={}
    
old_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
old_test_hyperparameters_df = pd.read_json(old_test_hyperparameters_file_name)
old_test_hyperparameters_df.to_json(f'{old_test_file_name[:-4]}_raw_df.json')

P.cut_off_freq=old_test_hyperparameters_df["cut_off_freq"][0]
min_key=P.cut_off_freq
print(f"cut_off_freq: {P.cut_off_freq}")

ffted_dfList_exp1 = get_ffted_dfList(raw_dfList_exp1, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

ffted_dfList_exp1_user_47 = get_ffted_dfList(raw_dfList_exp1_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)


dfList_dict={
            "dfList_exp1": ffted_dfList_exp1,
            "dfList_exp2": ffted_dfList_exp2,
            "dfList_exp1_user_47": ffted_dfList_exp1_user_47,
            "dfList_exp2_user_47": ffted_dfList_exp2_user_47
}
    

# #-------

EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=old_test_hyperparameters_df)
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"cut_off_freq: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
# #-------
# #-------
key_column= ["cut_off_freq"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)


EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
# #-------

reseting experiment params successful!
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
cut_off_freq: 37


  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  6.151549926027656
MakeWACAXExpDicUnknown Time:  32.82430716231465
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 4571.95it/s]

 20%|██        | 1/5 [00:05<00:22,  5.73s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 7321.18it/s]

 40%|████      | 2/5 [00:10<00:16,  5.42s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 8042.77it/s]

 60%|██████    | 3/5 [00:16<00:11,  5.56s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 9248.74it/s]

 80%|████████  | 4/5 [00:22<00:05,  5.64s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 7635.73it/s]

 11%|█         | 1/9 [01:07<09:02, 67.87s/it]

window_size: 250
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  2.9513608403503895
MakeWACAXExpDicUnknown Time:  16.479107059538364
Done extracting features


...


  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 9591.37it/s]

 20%|██        | 1/5 [00:06<00:26,  6.72s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 6465.71it/s]

 40%|████      | 2/5 [00:13<00:20,  6.71s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 9922.65it/s]

 60%|██████    | 3/5 [00:20<00:13,  6.72s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 11560.93it/s]

 80%|████████  | 4/5 [00:26<00:06,  6.68s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 6308.17it/s]

 78%|███████▊  | 7/9 [04:14<01:04, 32.05s/it]

window_size: 1750
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  0.7853096295148134
MakeWACAXExpDicUnknown Time:  3.342760931700468
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 10782.27it/s]

 20%|██        | 1/5 [00:04<00:18,  4.75s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 9385.33it/s]

 40%|████      | 2/5 [00:09<00:14,  4.79s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 7934.74it/s]

 60%|██████    | 3/5 [00:14<00:09,  4.76s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8100.24it/s]

 80%|████████  | 4/5 [00:19<00:04,  4.80s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 9506.58it/s]

 89%|████████▉ | 8/9 [04:42<00:30, 30.87s/it]

window_size: 2000
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  0.7706467797979712
MakeWACAXExpDicUnknown Time:  3.0806216718629003
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 11391.37it/s]

 20%|██        | 1/5 [00:06<00:26,  6.50s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 10541.10it/s]

 40%|████      | 2/5 [00:12<00:19,  6.48s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 7737.14it/s]

 60%|██████    | 3/5 [00:19<00:12,  6.33s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 6126.65it/s]

 80%|████████  | 4/5 [00:25<00:06,  6.28s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 8687.46it/s]

100%|██████████| 9/9 [05:18<00:00, 35.42s/it]


In [ ]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butterworth"

preprocessing_method="Naive"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{train_file_name}" + "-"*40 + "\n")
    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")
    





EER_df_train_dict={}
EER_df_test_dict={}
for clf_type in tqdm(RIVAL_CLASSIFIER_TYPE_LST):
    
    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_type}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)
    
    P.cut_off_freq=rival_test_hyperparameters_df["cut_off_freq"][0]
    print(f"cut_off_freq: {P.cut_off_freq}")

    ffted_dfList_exp1 = get_ffted_dfList(raw_dfList_exp1, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    ffted_dfList_exp1_user_47 = get_ffted_dfList(raw_dfList_exp1_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)


    dfList_dict={
                "dfList_exp1": ffted_dfList_exp1,
                "dfList_exp2": ffted_dfList_exp2,
                "dfList_exp1_user_47": ffted_dfList_exp1_user_47,
                "dfList_exp2_user_47": ffted_dfList_exp2_user_47
    }
    
    EER_df_train_dict[P.cut_off_freq] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                                    extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                                    param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\cut_off_freq: {P.cut_off_freq}\n")
        f.write(EER_df_train_dict[P.cut_off_freq].to_string())


        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
reseting experiment params successful!


  0%|          | 0/3 [00:00<?, ?it/s]

cut_off_freq: 41



  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 337
len_exp2_user_47: 289
MakeWACAXExpDicOwner Time:  14.4827835210599
MakeWACAXExpDicUnknown Time:  95.08045976795256
Done extracting features


...



  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 13637.80it/s]


 33%|███▎      | 1/3 [06:33<13:06, 393.02s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 14072.48it/s]


 67%|██████▋   | 2/3 [15:07<07:44, 464.62s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 10873.11it/s]


100%|██████████| 3/3 [23:10<00:00, 463.39s/it]


  0%|          | 0/5 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 18633.07it/s]


 20%|██        | 1/5 [00:11<00:47, 11.96s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 10057.08it/s]


 40%|████      | 2/5 [00:23<00:34, 11.48s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 17221.53it/s]


 60%|██████    | 3/5 [00:34<00:22, 11.37s/it]


run: 3: 100%|██████████| 20/20 [00:00<00:00, 10976.98it/s]


 80%|████████  | 4/5 [00:45<00:11, 11.45s/it]


run: 4: 100%|██████████| 20/20 [00:00<00:00, 10290.25it/s]


100%|██████████| 5/5 [00:57<00:00, 11.45s/it]

 22%|██▏       | 2/9 [58:56<3:20:42, 1720.40

window_size: 500
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 83
len_exp2_user_47: 71
MakeWACAXExpDicOwner Time:  4.4516099649481475
MakeWACAXExpDicUnknown Time:  25.269359302241355
Done extracting features




  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 19368.76it/s]


 33%|███▎      | 1/3 [07:11<14:22, 431.12s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 15842.51it/s]


 67%|██████▋   | 2/3 [15:16<07:42, 462.79s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 14413.42it/s]


100%|██████████| 3/3 [22:25<00:00, 448.39s/it]


  0%|          | 0/5 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 15009.14it/s]


 20%|██        | 1/5 [00:09<00:39,  9.85s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 12832.50it/s]


 40%|████      | 2/5 [00:19<00:28,  9.51s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 18067.22it/s]


 60%|██████    | 3/5 [00:28<00:18,  9.36s/it]


run: 3: 100%|██████████| 20/20 [00:00<00:00, 13543.12it/s]


 80%|████████  | 4/5 [00:37<00:09,  9.33s/it]


run: 4: 100%|██████████| 20/20 [00:00<00:00, 15709.00it/s]


100%|██████████| 5/5 [00:46<00:00,  9.36s/it]

 33%|███▎      | 3/9 [1:22:39<2:38:27, 1584.

window_size: 750
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 55
len_exp2_user_47: 47
MakeWACAXExpDicOwner Time:  3.5906891161575913
MakeWACAXExpDicUnknown Time:  18.114789451938123
Done extracting features




  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 13171.00it/s]


## 1.2 Realworld Approach (per unknown window application of filter)
### Optimizing and Testing

In [15]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butterworth"

preprocessing_method="Realworld-per_unknown_window"

time_of_execution = time.strftime("%Y-%m-%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/clip={P.scaler_clip}-Smoothing={P.smoothing}-Prep={preprocessing_method}-EER_df_test_dict.txt"
old_test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/old_{time_of_execution}_{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"



P.Butter_per_win_argdict={
    "filter_order": P.filter_order,
    "cut_off_freq": None,
    "sampling_freq": P.sampling_freq,
    "filtfilt": P.filtfilt,
    }


EER_df_test_dict={}

    
old_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
old_test_hyperparameters_df = pd.read_json(old_test_hyperparameters_file_name)
old_test_hyperparameters_df.to_json(f'{old_test_file_name[:-4]}_raw_df.json')

P.cut_off_freq=old_test_hyperparameters_df["cut_off_freq"][0]

P.Butter_per_win_argdict["cut_off_freq"]=P.cut_off_freq
min_key=P.cut_off_freq
print(f"cut_off_freq: {P.cut_off_freq}")


ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)


dfList_dict={
            "dfList_exp1": raw_dfList_exp1,
            "dfList_exp2": ffted_dfList_exp2,
            "dfList_exp1_user_47": raw_dfList_exp1_user_47,
            "dfList_exp2_user_47": ffted_dfList_exp2_user_47
}


EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=old_test_hyperparameters_df)
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

reseting experiment params successful!
train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
cut_off_freq: 49


  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  5.289517906494439
MakeWACAXExpDicUnknown Time:  36.922602359205484
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 10787.82it/s]

 20%|██        | 1/5 [00:05<00:21,  5.32s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 4365.88it/s]

 40%|████      | 2/5 [00:10<00:15,  5.18s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 6966.13it/s]

 60%|██████    | 3/5 [00:15<00:10,  5.11s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8160.12it/s]

 80%|████████  | 4/5 [00:20<00:05,  5.21s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 6205.51it/s]

 11%|█         | 1/9 [01:45<14:02, 105.29s/it]

window_size: 250
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  2.839411140419543
MakeWACAXExpDicUnknown Time:  18.91787970252335
Done extracting features


...


  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 6530.13it/s]

 20%|██        | 1/5 [00:03<00:14,  3.54s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 9461.55it/s]

 40%|████      | 2/5 [00:06<00:10,  3.42s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 5916.64it/s]

 60%|██████    | 3/5 [00:10<00:06,  3.37s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 6033.23it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.38s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 10951.19it/s]

 78%|███████▊  | 7/9 [05:15<01:06, 33.27s/it]

window_size: 1750
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  0.9026127699762583
MakeWACAXExpDicUnknown Time:  3.7978569390252233
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 11060.93it/s]

 20%|██        | 1/5 [00:03<00:12,  3.21s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 6198.17it/s]

 40%|████      | 2/5 [00:06<00:09,  3.27s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 14660.27it/s]

 60%|██████    | 3/5 [00:09<00:06,  3.28s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8147.44it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.28s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 8004.40it/s]

 89%|████████▉ | 8/9 [05:39<00:30, 30.31s/it]

window_size: 2000
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  0.8367584384977818
MakeWACAXExpDicUnknown Time:  3.655451947823167
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 11287.15it/s]

 20%|██        | 1/5 [00:06<00:25,  6.42s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 8653.40it/s]

 40%|████      | 2/5 [00:12<00:19,  6.42s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 7646.86it/s]

 60%|██████    | 3/5 [00:19<00:12,  6.37s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8380.23it/s]

 80%|████████  | 4/5 [00:25<00:06,  6.43s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 7839.82it/s]

100%|██████████| 9/9 [06:18<00:00, 42.09s/it]


In [ ]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butterworth"

preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    







P.Butter_per_win_argdict={
    "filter_order": P.filter_order,
    "cut_off_freq": None,
    "sampling_freq": P.sampling_freq,
    "filtfilt": P.filtfilt,
    }


EER_df_train_dict={}
EER_df_test_dict={}

for clf_type in tqdm(RIVAL_CLASSIFIER_TYPE_LST):
    
    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_type}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)
    
    P.cut_off_freq=rival_test_hyperparameters_df["cut_off_freq"][0]
    
    P.Butter_per_win_argdict["cut_off_freq"]=P.cut_off_freq
    print(f"cut_off_freq: {P.cut_off_freq}")

    
    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": ffted_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": ffted_dfList_exp2_user_47
    }
    

    EER_df_train_dict[P.cut_off_freq] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                                extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                                param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\cut_off_freq: {P.cut_off_freq}\n")
        f.write(EER_df_train_dict[P.cut_off_freq].to_string())



mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
reseting experiment params successful!


  0%|          | 0/3 [00:00<?, ?it/s]

cut_off_freq: 49



  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 337
len_exp2_user_47: 289
MakeWACAXExpDicOwner Time:  12.72940863110125


# 2. Butterworth frequency Cut-off + EMA span
## 2.1 Naive Approach
### Optimizing and Testing

In [16]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_2)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butter+EMA"

preprocessing_method="Naive"
time_of_execution = time.strftime("%Y-%m-%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/clip={P.scaler_clip}-Smoothing={P.smoothing}-Prep={preprocessing_method}-EER_df_test_dict.txt"
old_test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/old_{time_of_execution}_{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"




EER_df_test_dict={}

    
old_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
old_test_hyperparameters_df = pd.read_json(old_test_hyperparameters_file_name)
old_test_hyperparameters_df.to_json(f'{old_test_file_name[:-4]}_raw_df.json')

P.cut_off_freq=old_test_hyperparameters_df["cut_off_freq"][0]
P.span=old_test_hyperparameters_df["EMA_span"][0]

min_key= P.cut_off_freq, P.span
print(f"cut_off_freq: {P.cut_off_freq}, EMA span: {P.span}")


ffted_dfList_exp1 = get_ffted_dfList(raw_dfList_exp1, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

EMAed_dfList_exp1 = get_EMAed_dfList(ffted_dfList_exp1, span=P.span)
EMAed_dfList_exp2 = get_EMAed_dfList(ffted_dfList_exp2, span=P.span)

ffted_dfList_exp1_user_47 = get_ffted_dfList(raw_dfList_exp1_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

EMAed_dfList_exp1_user_47 = get_EMAed_dfList(ffted_dfList_exp1_user_47, span=P.span)
EMAed_dfList_exp2_user_47 = get_EMAed_dfList(ffted_dfList_exp2_user_47, span=P.span)

dfList_dict={
            "dfList_exp1": EMAed_dfList_exp1,
            "dfList_exp2": EMAed_dfList_exp2,
            "dfList_exp1_user_47": EMAed_dfList_exp1_user_47,
            "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
}
    

EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=old_test_hyperparameters_df)
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq", "EMA_span"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

reseting experiment params successful!
train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
cut_off_freq: 41, EMA span: 48


  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  5.327125420793891
MakeWACAXExpDicUnknown Time:  32.231868176721036
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 9523.85it/s]

 20%|██        | 1/5 [00:05<00:20,  5.24s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 6277.02it/s]

 40%|████      | 2/5 [00:10<00:15,  5.25s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 7247.80it/s]

 60%|██████    | 3/5 [00:15<00:10,  5.27s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 7495.18it/s]

 80%|████████  | 4/5 [00:21<00:05,  5.42s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 9070.73it/s]

 11%|█         | 1/9 [01:05<08:45, 65.66s/it]

window_size: 250
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  2.8676016042008996
MakeWACAXExpDicUnknown Time:  16.45726869441569
Done extracting features


...


  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 11187.79it/s]

 20%|██        | 1/5 [00:03<00:13,  3.50s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 7945.26it/s]

 40%|████      | 2/5 [00:06<00:10,  3.42s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 10060.70it/s]

 60%|██████    | 3/5 [00:10<00:06,  3.40s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 7693.15it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.37s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 5402.94it/s]

 78%|███████▊  | 7/9 [03:55<00:52, 26.49s/it]

window_size: 1750
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  0.7731208503246307
MakeWACAXExpDicUnknown Time:  3.1045133247971535
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 8729.04it/s]

 20%|██        | 1/5 [00:03<00:13,  3.32s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 8395.32it/s]

 40%|████      | 2/5 [00:06<00:09,  3.32s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 16987.87it/s]

 60%|██████    | 3/5 [00:09<00:06,  3.25s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 5649.66it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.27s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 9788.34it/s]

 89%|████████▉ | 8/9 [04:15<00:24, 24.58s/it]

window_size: 2000
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  0.7052394282072783
MakeWACAXExpDicUnknown Time:  2.8162678508087993
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 12343.45it/s]

 20%|██        | 1/5 [00:03<00:13,  3.30s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 8523.28it/s]

 40%|████      | 2/5 [00:06<00:09,  3.29s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 4470.59it/s]

 60%|██████    | 3/5 [00:09<00:06,  3.23s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 6946.51it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.27s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 4680.62it/s]

100%|██████████| 9/9 [04:35<00:00, 30.66s/it]


In [21]:
P.smoothing = "Butter+EMA" 
preprocessing_method="Naive"

rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/WACA-kNN/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

P.cut_off_freq=rival_test_hyperparameters_df["cut_off_freq"][0]
P.span=rival_test_hyperparameters_df["EMA_span"][0]

print(f"cut_off_freq: {P.cut_off_freq}, EMA span: {P.span}")
rival_test_hyperparameters_df

cut_off_freq: 47, EMA span: 41


,window_size,step_width,Mean_EER,best_n_neighbors,cut_off_freq,EMA_span
0,125,62,0.204121,2,47,41
1,250,125,0.168393,4,47,41
2,500,250,0.150881,9,47,41
3,750,375,0.157118,19,47,41
4,1000,500,0.148969,13,47,41
5,1250,625,0.160548,19,47,41
6,1500,750,0.158079,19,47,41
7,1750,875,0.161596,19,47,41
8,2000,1000,0.164788,19,47,41


In [ ]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_2)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butter+EMA"

preprocessing_method="Naive"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    
indices = list(range(1, 50))
mesh = np.array(np.meshgrid(indices, indices))
index_pairs = mesh.T.reshape(-1, 2)

print(f"total cut_off_span_pairs: {index_pairs.shape}, choice_num: {CHOICE_NUM_PAIRS}")
cut_off_span_pairs = index_pairs[np.random.choice(index_pairs.shape[0], size=CHOICE_NUM_PAIRS, replace=False), :]








EER_df_train_dict={}
EER_df_test_dict={}

for clf_type in tqdm(RIVAL_CLASSIFIER_TYPE_LST):
    
    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_type}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.cut_off_freq=rival_test_hyperparameters_df["cut_off_freq"][0]
    P.span=rival_test_hyperparameters_df["EMA_span"][0]

    key_pair= P.cut_off_freq, P.span, clf_type
    print(f"cut_off_freq: {P.cut_off_freq}, EMA span: {P.span}")

    
    ffted_dfList_exp1 = get_ffted_dfList(raw_dfList_exp1, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    EMAed_dfList_exp1 = get_EMAed_dfList(ffted_dfList_exp1, span=P.span)
    EMAed_dfList_exp2 = get_EMAed_dfList(ffted_dfList_exp2, span=P.span)
    
    ffted_dfList_exp1_user_47 = get_ffted_dfList(raw_dfList_exp1_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    EMAed_dfList_exp1_user_47 = get_EMAed_dfList(ffted_dfList_exp1_user_47, span=P.span)
    EMAed_dfList_exp2_user_47 = get_EMAed_dfList(ffted_dfList_exp2_user_47, span=P.span)
    
    dfList_dict={
                "dfList_exp1": EMAed_dfList_exp1,
                "dfList_exp2": EMAed_dfList_exp2,
                "dfList_exp1_user_47": EMAed_dfList_exp1_user_47,
                "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[key_pair] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                      extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                      param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\cut_off_freq, span: {key_pair}\n")
        f.write(EER_df_train_dict[key_pair].to_string())



        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key[:-1]] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key[:-1]].to_string())
#-------
#-------
key_column= ["cut_off_freq", "EMA_span"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")
        

reseting experiment params successful!
train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
total cut_off_span_pairs: (2401, 2), choice_num: 13


  0%|          | 0/3 [00:00<?, ?it/s]

cut_off_freq: 24, EMA span: 30



  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 337
len_exp2_user_47: 289
MakeWACAXExpDicOwner Time:  13.077337847091258
MakeWACAXExpDicUnknown Time:  68.97748926375061
Done extracting features


...



  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 18375.92it/s]


 33%|███▎      | 1/3 [06:29<12:59, 389.58s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 14621.94it/s]


 67%|██████▋   | 2/3 [12:25<06:09, 369.84s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 11572.09it/s]


100%|██████████| 3/3 [17:54<00:00, 358.22s/it]


  0%|          | 0/5 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 28542.39it/s]


 20%|██        | 1/5 [00:06<00:25,  6.31s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 15082.00it/s]


 40%|████      | 2/5 [00:12<00:18,  6.17s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 15185.75it/s]


 60%|██████    | 3/5 [00:18<00:12,  6.01s/it]


run: 3: 100%|██████████| 20/20 [00:00<00:00, 13727.06it/s]


 80%|████████  | 4/5 [00:24<00:05,  5.99s/it]


run: 4: 100%|██████████| 20/20 [00:00<00:00, 20743.34it/s]


100%|██████████| 5/5 [00:30<00:00,  6.01s/it]

 67%|██████▋   | 6/9 [1:57:51<56:03, 1121.19

window_size: 1500
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 27
len_exp2_user_47: 23
MakeWACAXExpDicOwner Time:  2.4641578728333116
MakeWACAXExpDicUnknown Time:  7.697064138017595
Done extracting features




  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 24463.72it/s]


 33%|███▎      | 1/3 [05:20<10:41, 320.63s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 16457.93it/s]


 67%|██████▋   | 2/3 [10:30<05:14, 314.05s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 15866.48it/s]


100%|██████████| 3/3 [15:55<00:00, 318.51s/it]


  0%|          | 0/5 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 17349.76it/s]


 20%|██        | 1/5 [00:06<00:24,  6.16s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 21631.27it/s]


 40%|████      | 2/5 [00:12<00:18,  6.09s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 12830.54it/s]


 60%|██████    | 3/5 [00:17<00:11,  5.95s/it]


run: 3: 100%|██████████| 20/20 [00:00<00:00, 22086.91it/s]


 80%|████████  | 4/5 [00:23<00:05,  5.91s/it]


run: 4: 100%|██████████| 20/20 [00:00<00:00, 14339.50it/s]


100%|██████████| 5/5 [00:29<00:00,  5.93s/it]

 78%|███████▊  | 7/9 [2:14:26<36:00, 1080.17

window_size: 1750
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 23
len_exp2_user_47: 19
MakeWACAXExpDicOwner Time:  2.142112142406404
MakeWACAXExpDicUnknown Time:  6.89364912873134
Done extracting features




  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 11319.13it/s]


 33%|███▎      | 1/3 [05:19<10:38, 319.19s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 16313.90it/s]


 67%|██████▋   | 2/3 [10:25<05:11, 311.74s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 25930.78it/s]


## 2.2 Realworld Approach (per unknown window application of filter)
### Optimizing and Testing

In [17]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_2)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butter+EMA"


preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y-%m-%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/clip={P.scaler_clip}-Smoothing={P.smoothing}-Prep={preprocessing_method}-EER_df_test_dict.txt"
old_test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/old_{time_of_execution}_{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"



P.Butter_per_win_argdict={
    "filter_order": P.filter_order,
    "cut_off_freq": None,
    "sampling_freq": P.sampling_freq,
    "filtfilt": P.filtfilt,
    }

EER_df_test_dict={}


old_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
old_test_hyperparameters_df = pd.read_json(old_test_hyperparameters_file_name)
old_test_hyperparameters_df.to_json(f'{old_test_file_name[:-4]}_raw_df.json')

P.cut_off_freq=old_test_hyperparameters_df["cut_off_freq"][0]
P.span=old_test_hyperparameters_df["EMA_span"][0]

P.Butter_per_win_argdict["cut_off_freq"]=P.cut_off_freq
P.EMA_per_win_span=P.span

min_key= P.cut_off_freq, P.span
print(f"cut_off_freq: {P.cut_off_freq}, EMA span: {P.span}")


ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
EMAed_dfList_exp2 = get_EMAed_dfList(ffted_dfList_exp2, span=P.span)

ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
EMAed_dfList_exp2_user_47 = get_EMAed_dfList(ffted_dfList_exp2_user_47, span=P.span)

dfList_dict={
            "dfList_exp1": raw_dfList_exp1,
            "dfList_exp2": EMAed_dfList_exp2,
            "dfList_exp1_user_47": raw_dfList_exp1_user_47,
            "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
}



EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=old_test_hyperparameters_df)
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq", "EMA_span"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

reseting experiment params successful!
train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
cut_off_freq: 40, EMA span: 3


  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  5.200552448630333
MakeWACAXExpDicUnknown Time:  36.935267420485616
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 10302.88it/s]

 20%|██        | 1/5 [00:05<00:20,  5.20s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 8013.57it/s]

 40%|████      | 2/5 [00:10<00:15,  5.23s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 7520.72it/s]

 60%|██████    | 3/5 [00:15<00:10,  5.15s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8016.64it/s]

 80%|████████  | 4/5 [00:20<00:05,  5.27s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 9379.03it/s]

 11%|█         | 1/9 [01:53<15:05, 113.21s/it]

window_size: 250
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  2.906321233138442
MakeWACAXExpDicUnknown Time:  18.975488111376762
Done extracting features


...


  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 9670.98it/s]

 20%|██        | 1/5 [00:03<00:14,  3.63s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 9606.74it/s]

 40%|████      | 2/5 [00:06<00:10,  3.44s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 5277.18it/s]

 60%|██████    | 3/5 [00:10<00:06,  3.39s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 5573.83it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.36s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 8158.54it/s]

 78%|███████▊  | 7/9 [05:36<01:09, 34.67s/it]

window_size: 1750
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  0.8518248479813337
MakeWACAXExpDicUnknown Time:  3.5841895388439298
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 5678.72it/s]

 20%|██        | 1/5 [00:06<00:25,  6.37s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 8297.34it/s]

 40%|████      | 2/5 [00:12<00:18,  6.31s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 7419.61it/s]

 60%|██████    | 3/5 [00:19<00:12,  6.33s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 6908.75it/s]

 80%|████████  | 4/5 [00:25<00:06,  6.26s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 11628.23it/s]

 89%|████████▉ | 8/9 [06:16<00:36, 36.12s/it]

window_size: 2000
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  0.7651711134240031
MakeWACAXExpDicUnknown Time:  3.165899026207626
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 9204.09it/s]

 20%|██        | 1/5 [00:03<00:12,  3.25s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 8451.15it/s]

 40%|████      | 2/5 [00:06<00:09,  3.28s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 7701.62it/s]

 60%|██████    | 3/5 [00:09<00:06,  3.25s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 6189.94it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.26s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 8405.42it/s]

100%|██████████| 9/9 [06:39<00:00, 44.38s/it]


In [ ]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_2)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butter+EMA"


preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    
indices = list(range(1, 50))
mesh = np.array(np.meshgrid(indices, indices))
index_pairs = mesh.T.reshape(-1, 2)

print(f"total cut_off_span_pairs: {index_pairs.shape}, choice_num: {CHOICE_NUM_PAIRS}")
cut_off_span_pairs = index_pairs[np.random.choice(index_pairs.shape[0], size=CHOICE_NUM_PAIRS, replace=False), :]






P.Butter_per_win_argdict={
    "filter_order": P.filter_order,
    "cut_off_freq": None,
    "sampling_freq": P.sampling_freq,
    "filtfilt": P.filtfilt,
    }


EER_df_train_dict={}
EER_df_test_dict={}

for clf_type in tqdm(RIVAL_CLASSIFIER_TYPE_LST):

    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_type}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.cut_off_freq=rival_test_hyperparameters_df["cut_off_freq"][0]
    P.span=rival_test_hyperparameters_df["EMA_span"][0]

    P.Butter_per_win_argdict["cut_off_freq"]=P.cut_off_freq
    P.EMA_per_win_span=P.span

    key_pair= P.cut_off_freq, P.span, clf_type
    print(f"cut_off_freq: {P.cut_off_freq}, EMA span: {P.span}")


    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    EMAed_dfList_exp2 = get_EMAed_dfList(ffted_dfList_exp2, span=P.span)
    
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    EMAed_dfList_exp2_user_47 = get_EMAed_dfList(ffted_dfList_exp2_user_47, span=P.span)
    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": EMAed_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[key_pair] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                      extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                      param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\cut_off_freq, span: {key_pair}\n")
        f.write(EER_df_train_dict[key_pair].to_string())



        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key[:-1]] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key[:-1]].to_string())
#-------
#-------
key_column= ["cut_off_freq", "EMA_span"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

# 3. EMA span
## 3.1 Naive Approach
### Optimizing and Testing

In [18]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "EMA"


preprocessing_method="Naive"
time_of_execution = time.strftime("%Y-%m-%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/clip={P.scaler_clip}-Smoothing={P.smoothing}-Prep={preprocessing_method}-EER_df_test_dict.txt"
old_test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/old_{time_of_execution}_{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"



EER_df_test_dict={}
    
old_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
old_test_hyperparameters_df = pd.read_json(old_test_hyperparameters_file_name)
old_test_hyperparameters_df.to_json(f'{old_test_file_name[:-4]}_raw_df.json')

P.span=old_test_hyperparameters_df["EMA_span"][0]
min_key=P.span
print(f"EMA span: {P.span}")


EMAed_dfList_exp1 = get_EMAed_dfList(raw_dfList_exp1, span=P.span)
EMAed_dfList_exp2 = get_EMAed_dfList(raw_dfList_exp2, span=P.span)

EMAed_dfList_exp1_user_47 = get_EMAed_dfList(raw_dfList_exp1_user_47, span=P.span)
EMAed_dfList_exp2_user_47 = get_EMAed_dfList(raw_dfList_exp2_user_47, span=P.span)


dfList_dict={
            "dfList_exp1": EMAed_dfList_exp1,
            "dfList_exp2": EMAed_dfList_exp2,
            "dfList_exp1_user_47": EMAed_dfList_exp1_user_47,
            "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
}


EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=old_test_hyperparameters_df)
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["EMA_span"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

reseting experiment params successful!
train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
EMA span: 37


  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  5.504320846870542
MakeWACAXExpDicUnknown Time:  32.04493124131113
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 11335.96it/s]

 20%|██        | 1/5 [00:05<00:21,  5.34s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 7152.63it/s]

 40%|████      | 2/5 [00:10<00:16,  5.36s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 7674.85it/s]

 60%|██████    | 3/5 [00:16<00:10,  5.41s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8756.38it/s]

 80%|████████  | 4/5 [00:21<00:05,  5.32s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 7672.04it/s]

 11%|█         | 1/9 [01:05<08:44, 65.51s/it]

window_size: 250
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  2.893535257317126
MakeWACAXExpDicUnknown Time:  16.49389768857509
Done extracting features


...


  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 10361.42it/s]

 20%|██        | 1/5 [00:03<00:13,  3.34s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 8655.19it/s]

 40%|████      | 2/5 [00:06<00:09,  3.30s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 4862.96it/s]

 60%|██████    | 3/5 [00:10<00:06,  3.40s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 7219.11it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.37s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 10580.99it/s]

 78%|███████▊  | 7/9 [03:44<00:51, 25.76s/it]

window_size: 1750
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  0.8082584841176867
MakeWACAXExpDicUnknown Time:  3.1096847830340266
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 10488.38it/s]

 20%|██        | 1/5 [00:03<00:13,  3.33s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 8729.04it/s]

 40%|████      | 2/5 [00:06<00:10,  3.35s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 7747.14it/s]

 60%|██████    | 3/5 [00:10<00:06,  3.34s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8528.47it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.32s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 6011.62it/s]

 89%|████████▉ | 8/9 [04:05<00:24, 24.23s/it]

window_size: 2000
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  0.7101543862372637
MakeWACAXExpDicUnknown Time:  2.7811973309144378
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 8055.13it/s]

 20%|██        | 1/5 [00:06<00:25,  6.34s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 11000.01it/s]

 40%|████      | 2/5 [00:12<00:19,  6.39s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 8267.90it/s]

 60%|██████    | 3/5 [00:19<00:12,  6.38s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 7173.43it/s]

 80%|████████  | 4/5 [00:25<00:06,  6.27s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 15006.45it/s]

100%|██████████| 9/9 [04:40<00:00, 31.14s/it]


In [18]:
rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_type}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)
rival_test_hyperparameters_df
print(rival_test_hyperparameters_file_name)
print(RIVAL_CLASSIFIER_TYPE_LST)

final_experiments_results/WACA-LOF/EMA_Mean_EER_Naive_df_test_dict_raw_df.json
['OCSVM', 'kNN', 'LOF']


In [ ]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "EMA"


preprocessing_method="Naive"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    





EER_df_train_dict={}
EER_df_test_dict={}
for clf_type in tqdm(RIVAL_CLASSIFIER_TYPE_LST):
    
    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_type}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.span=rival_test_hyperparameters_df["EMA_span"][0]
    print(f"EMA span: {P.span}")

    
    EMAed_dfList_exp1 = get_EMAed_dfList(raw_dfList_exp1, span=P.span)
    EMAed_dfList_exp2 = get_EMAed_dfList(raw_dfList_exp2, span=P.span)
    
    EMAed_dfList_exp1_user_47 = get_EMAed_dfList(raw_dfList_exp1_user_47, span=P.span)
    EMAed_dfList_exp2_user_47 = get_EMAed_dfList(raw_dfList_exp2_user_47, span=P.span)

    
    dfList_dict={
                "dfList_exp1": EMAed_dfList_exp1,
                "dfList_exp2": EMAed_dfList_exp2,
                "dfList_exp1_user_47": EMAed_dfList_exp1_user_47,
                "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[P.span] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                    extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                    param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\EMA span: {P.span}\n")
        f.write(EER_df_train_dict[P.span].to_string())


        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["EMA_span"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
reseting experiment params successful!


  0%|          | 0/3 [00:00<?, ?it/s]

EMA span: 25



  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 337
len_exp2_user_47: 289
MakeWACAXExpDicOwner Time:  13.062254984863102
MakeWACAXExpDicUnknown Time:  70.15507365902886
Done extracting features


...



  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 18575.31it/s]


 33%|███▎      | 1/3 [06:24<12:49, 384.78s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 12274.81it/s]


 67%|██████▋   | 2/3 [12:31<06:14, 374.38s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 13586.99it/s]


100%|██████████| 3/3 [19:05<00:00, 381.71s/it]


  0%|          | 0/5 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 18880.50it/s]


 20%|██        | 1/5 [00:06<00:27,  6.75s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 23373.11it/s]


 40%|████      | 2/5 [00:12<00:19,  6.35s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 11642.76it/s]


 60%|██████    | 3/5 [00:19<00:12,  6.38s/it]


run: 3: 100%|██████████| 20/20 [00:00<00:00, 14480.59it/s]


 80%|████████  | 4/5 [00:25<00:06,  6.27s/it]


run: 4: 100%|██████████| 20/20 [00:00<00:00, 14011.37it/s]


100%|██████████| 5/5 [00:31<00:00,  6.30s/it]

 22%|██▏       | 2/9 [46:01<2:37:49, 1352.73

window_size: 500
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 83
len_exp2_user_47: 71
MakeWACAXExpDicOwner Time:  4.32451828988269
MakeWACAXExpDicUnknown Time:  19.267858140170574
Done extracting features




  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 17795.10it/s]


 33%|███▎      | 1/3 [05:48<11:37, 348.95s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 16269.60it/s]


 67%|██████▋   | 2/3 [11:24<05:40, 340.90s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 14055.98it/s]


100%|██████████| 3/3 [17:20<00:00, 346.80s/it]


  0%|          | 0/5 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 17288.97it/s]


 20%|██        | 1/5 [00:07<00:29,  7.44s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 14844.47it/s]


 40%|████      | 2/5 [00:14<00:21,  7.22s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 14657.71it/s]


 60%|██████    | 3/5 [00:21<00:14,  7.16s/it]


run: 3: 100%|██████████| 20/20 [00:00<00:00, 14883.97it/s]


 80%|████████  | 4/5 [00:28<00:07,  7.13s/it]


run: 4: 100%|██████████| 20/20 [00:00<00:00, 14413.42it/s]


100%|██████████| 5/5 [00:35<00:00,  7.16s/it]

 33%|███▎      | 3/9 [1:04:22<2:03:45, 1237.

window_size: 750
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 55
len_exp2_user_47: 47
MakeWACAXExpDicOwner Time:  3.3790314039215446
MakeWACAXExpDicUnknown Time:  13.367395809851587
Done extracting features




  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 8494.79it/s]


 33%|███▎      | 1/3 [05:35<11:11, 335.68s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 14354.22it/s]


 67%|██████▋   | 2/3 [10:59<05:28, 328.55s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 13247.96it/s]


## 3.2 Realworld Approach (per unknown window application of filter)
### Optimizing and Testing

In [19]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "EMA"



preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y-%m-%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/clip={P.scaler_clip}-Smoothing={P.smoothing}-Prep={preprocessing_method}-EER_df_test_dict.txt"
old_test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/old_{time_of_execution}_{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
    

EER_df_test_dict={}
    
old_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
old_test_hyperparameters_df = pd.read_json(old_test_hyperparameters_file_name)
old_test_hyperparameters_df.to_json(f'{old_test_file_name[:-4]}_raw_df.json')

P.span=old_test_hyperparameters_df["EMA_span"][0]
P.EMA_per_win_span=P.span
min_key=P.span
print(f"EMA span: {P.span}")


EMAed_dfList_exp2 = get_EMAed_dfList(raw_dfList_exp2, span=P.span)

EMAed_dfList_exp2_user_47 = get_EMAed_dfList(raw_dfList_exp2_user_47, span=P.span)


dfList_dict={
            "dfList_exp1": raw_dfList_exp1,
            "dfList_exp2": EMAed_dfList_exp2,
            "dfList_exp1_user_47": raw_dfList_exp1_user_47,
            "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
}


EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=old_test_hyperparameters_df)
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["EMA_span"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

reseting experiment params successful!
train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
EMA span: 9


  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  5.268382767215371
MakeWACAXExpDicUnknown Time:  36.96340117137879
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 15180.25it/s]

 20%|██        | 1/5 [00:03<00:12,  3.03s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 5327.45it/s]

 40%|████      | 2/5 [00:05<00:08,  2.97s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 7891.45it/s]

 60%|██████    | 3/5 [00:08<00:05,  2.95s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 5542.16it/s]

 80%|████████  | 4/5 [00:11<00:02,  2.98s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 14624.49it/s]

 11%|█         | 1/9 [01:05<08:40, 65.02s/it]

window_size: 250
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  2.8611738523468375
MakeWACAXExpDicUnknown Time:  18.98926841188222
Done extracting features


...


  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 11155.06it/s]

 20%|██        | 1/5 [00:03<00:14,  3.54s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 8994.86it/s]

 40%|████      | 2/5 [00:06<00:10,  3.42s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 7744.28it/s]

 60%|██████    | 3/5 [00:10<00:06,  3.38s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8867.45it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.39s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 8512.90it/s]

 78%|███████▊  | 7/9 [03:58<00:54, 27.27s/it]

window_size: 1750
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  0.7845384990796447
MakeWACAXExpDicUnknown Time:  3.4029627200216055
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 14852.35it/s]

 20%|██        | 1/5 [00:06<00:25,  6.35s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 7870.71it/s]

 40%|████      | 2/5 [00:12<00:19,  6.35s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 8292.42it/s]

 60%|██████    | 3/5 [00:18<00:12,  6.32s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 7748.58it/s]

 80%|████████  | 4/5 [00:25<00:06,  6.35s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 16690.43it/s]

 89%|████████▉ | 8/9 [04:35<00:30, 30.22s/it]

window_size: 2000
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  0.7646766528487206
MakeWACAXExpDicUnknown Time:  3.0294679468497634
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 10036.62it/s]

 20%|██        | 1/5 [00:03<00:13,  3.26s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 8292.42it/s]

 40%|████      | 2/5 [00:06<00:10,  3.37s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 7973.96it/s]

 60%|██████    | 3/5 [00:09<00:06,  3.30s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 5681.03it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.26s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 8328.64it/s]

100%|██████████| 9/9 [04:56<00:00, 32.92s/it]


In [ ]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "EMA"



preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    




EER_df_train_dict={}
EER_df_test_dict={}
for clf_type in tqdm(RIVAL_CLASSIFIER_TYPE_LST):
    
    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_type}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.span=rival_test_hyperparameters_df["EMA_span"][0]
    print(f"EMA span: {P.span}")

    P.EMA_per_win_span=P.span

    EMAed_dfList_exp2 = get_EMAed_dfList(raw_dfList_exp2, span=P.span)
    
    EMAed_dfList_exp2_user_47 = get_EMAed_dfList(raw_dfList_exp2_user_47, span=P.span)

    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": EMAed_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[P.span] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                    extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                    param_dist=param_dist) 
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\EMA span: {P.span}\n")
        f.write(EER_df_train_dict[P.span].to_string())


        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["EMA_span"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
reseting experiment params successful!


  0%|          | 0/3 [00:00<?, ?it/s]

EMA span: 9



  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 337
len_exp2_user_47: 289
MakeWACAXExpDicOwner Time:  13.175995624158531
MakeWACAXExpDicUnknown Time:  87.51563867088407
Done extracting features


...


  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 337
len_exp2_user_47: 289
MakeWACAXExpDicOwner Time:  12.653837237041444
MakeWACAXExpDicUnknown Time:  89.56300481688231
Done extracting features




  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 17147.60it/s]


 33%|███▎      | 1/3 [07:45<15:31, 465.96s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 12580.40it/s]


 67%|██████▋   | 2/3 [15:09<07:32, 452.49s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 18157.16it/s]


100%|██████████| 3/3 [23:02<00:00, 460.82s/it]


  0%|          | 0/5 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 17509.10it/s]


 20%|██        | 1/5 [00:09<00:37,  9.34s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 20345.88it/s]


 40%|████      | 2/5 [00:18<00:27,  9.18s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 21361.36it/s]


 60%|██████    | 3/5 [00:27<00:18,  9.31s/it]


run: 3: 100%|██████████| 20/20 [00:00<00:00, 15488.57it/s]


 80%|████████  | 4/5 [00:37<00:09,  9.27s/it]


run: 4: 100%|██████████| 20/20 [00:00<00:00, 21307.11it/s]


100%|██████████| 5/5 [00:46<00:00,  9.24s/it]

 11%|█         | 1/9 [25:46<3:26:12, 1546.58

window_size: 250
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 167
len_exp2_user_47: 143
MakeWACAXExpDicOwner Time:  7.051525607705116
MakeWACAXExpDicUnknown Time:  44.71043225098401
Done extracting features




  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 13595.80it/s]


 33%|███▎      | 1/3 [06:23<12:47, 383.78s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 15139.16it/s]


# 4. SMA winsize
## 4.1 Naive Approach
### Optimizing and Testing

In [20]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "SMA"


preprocessing_method="Naive"
time_of_execution = time.strftime("%Y-%m-%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/clip={P.scaler_clip}-Smoothing={P.smoothing}-Prep={preprocessing_method}-EER_df_test_dict.txt"
old_test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/old_{time_of_execution}_{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"


EER_df_test_dict={}

old_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
old_test_hyperparameters_df = pd.read_json(old_test_hyperparameters_file_name)
old_test_hyperparameters_df.to_json(f'{old_test_file_name[:-4]}_raw_df.json')

P.winsize=old_test_hyperparameters_df["SMA_winsize"][0]
min_key=P.winsize
print(f"SMA winsize: {P.winsize}")


SMAed_dfList_exp1 = get_SMAed_dfList(raw_dfList_exp1, winsize=P.winsize)
SMAed_dfList_exp2 = get_SMAed_dfList(raw_dfList_exp2, winsize=P.winsize)

SMAed_dfList_exp1_user_47 = get_SMAed_dfList(raw_dfList_exp1_user_47, winsize=P.winsize)
SMAed_dfList_exp2_user_47 = get_SMAed_dfList(raw_dfList_exp2_user_47, winsize=P.winsize)


dfList_dict={
            "dfList_exp1": SMAed_dfList_exp1,
            "dfList_exp2": SMAed_dfList_exp2,
            "dfList_exp1_user_47": SMAed_dfList_exp1_user_47,
            "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
}
    

EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=old_test_hyperparameters_df)
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["SMA_winsize"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

reseting experiment params successful!
train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
SMA winsize: 41


  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  5.588299156166613
MakeWACAXExpDicUnknown Time:  31.71025293599814
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 10656.26it/s]

 20%|██        | 1/5 [00:05<00:20,  5.11s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 7461.85it/s]

 40%|████      | 2/5 [00:10<00:15,  5.24s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 9722.54it/s]

 60%|██████    | 3/5 [00:15<00:10,  5.28s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 7004.52it/s]

 80%|████████  | 4/5 [00:21<00:05,  5.29s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 8403.73it/s]

 11%|█         | 1/9 [01:05<08:41, 65.17s/it]

window_size: 250
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  3.291580139659345
MakeWACAXExpDicUnknown Time:  16.227446667850018
Done extracting features


...


  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 10832.40it/s]

 20%|██        | 1/5 [00:03<00:13,  3.32s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 8468.21it/s]

 40%|████      | 2/5 [00:06<00:10,  3.39s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 9523.85it/s]

 60%|██████    | 3/5 [00:10<00:06,  3.37s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8478.48it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.36s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 6383.05it/s]

 78%|███████▊  | 7/9 [03:46<00:52, 26.33s/it]

window_size: 1750
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  1.199315662495792
MakeWACAXExpDicUnknown Time:  3.3917203564196825
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 7758.61it/s]

 20%|██        | 1/5 [00:03<00:12,  3.23s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 7162.40it/s]

 40%|████      | 2/5 [00:06<00:09,  3.23s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 8115.91it/s]

 60%|██████    | 3/5 [00:09<00:06,  3.24s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8203.22it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.31s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 8333.61it/s]

 89%|████████▉ | 8/9 [04:07<00:24, 24.69s/it]

window_size: 2000
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  1.1780718741938472
MakeWACAXExpDicUnknown Time:  3.2216739216819406
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 11158.03it/s]

 20%|██        | 1/5 [00:06<00:25,  6.31s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 8632.03it/s]

 40%|████      | 2/5 [00:12<00:19,  6.38s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 7681.88it/s]

 60%|██████    | 3/5 [00:19<00:12,  6.33s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8487.06it/s]

 80%|████████  | 4/5 [00:25<00:06,  6.32s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 6805.62it/s]

100%|██████████| 9/9 [04:43<00:00, 31.54s/it]


In [ ]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "SMA"



preprocessing_method="Naive"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    





EER_df_train_dict={}
EER_df_test_dict={}
for clf_type in tqdm(RIVAL_CLASSIFIER_TYPE_LST):

    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_type}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.winsize=rival_test_hyperparameters_df["SMA_winsize"][0]
    print(f"SMA winsize: {P.winsize}")


    SMAed_dfList_exp1 = get_SMAed_dfList(raw_dfList_exp1, winsize=P.winsize)
    SMAed_dfList_exp2 = get_SMAed_dfList(raw_dfList_exp2, winsize=P.winsize)
    
    SMAed_dfList_exp1_user_47 = get_SMAed_dfList(raw_dfList_exp1_user_47, winsize=P.winsize)
    SMAed_dfList_exp2_user_47 = get_SMAed_dfList(raw_dfList_exp2_user_47, winsize=P.winsize)

    
    dfList_dict={
                "dfList_exp1": SMAed_dfList_exp1,
                "dfList_exp2": SMAed_dfList_exp2,
                "dfList_exp1_user_47": SMAed_dfList_exp1_user_47,
                "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[P.winsize] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                       extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                       param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\SMA winsize: {P.winsize}\n")
        f.write(EER_df_train_dict[P.winsize].to_string())


        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["SMA_winsize"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

## 4.2 Realworld Approach (per unknown window application of filter)
### Optimizing and Testing

In [21]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "SMA"



preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y-%m-%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/clip={P.scaler_clip}-Smoothing={P.smoothing}-Prep={preprocessing_method}-EER_df_test_dict.txt"
old_test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/old_{time_of_execution}_{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"


EER_df_test_dict={}

old_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
old_test_hyperparameters_df = pd.read_json(old_test_hyperparameters_file_name)
old_test_hyperparameters_df.to_json(f'{old_test_file_name[:-4]}_raw_df.json')

P.winsize=old_test_hyperparameters_df["SMA_winsize"][0]
P.SMA_per_win_winsize=P.winsize
min_key=P.winsize
print(f"SMA winsize: {P.winsize}")


SMAed_dfList_exp2 = get_SMAed_dfList(raw_dfList_exp2, winsize=P.winsize)

SMAed_dfList_exp2_user_47 = get_SMAed_dfList(raw_dfList_exp2_user_47, winsize=P.winsize)


dfList_dict={
            "dfList_exp1": raw_dfList_exp1,
            "dfList_exp2": SMAed_dfList_exp2,
            "dfList_exp1_user_47": raw_dfList_exp1_user_47,
            "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
}


EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=old_test_hyperparameters_df)
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["SMA_winsize"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

reseting experiment params successful!
train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
SMA winsize: 1


  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  5.772508081048727
MakeWACAXExpDicUnknown Time:  36.227119009941816
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 9657.62it/s]

 20%|██        | 1/5 [00:03<00:12,  3.17s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 11055.10it/s]

 40%|████      | 2/5 [00:06<00:09,  3.04s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 8907.00it/s]

 60%|██████    | 3/5 [00:09<00:05,  2.98s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 7658.03it/s]

 80%|████████  | 4/5 [00:11<00:02,  2.95s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 8413.85it/s]

 11%|█         | 1/9 [01:03<08:25, 63.18s/it]

window_size: 250
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  3.032670099288225
MakeWACAXExpDicUnknown Time:  18.752352327108383
Done extracting features


...


  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 12838.40it/s]

 20%|██        | 1/5 [00:03<00:13,  3.41s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 7919.76it/s]

 40%|████      | 2/5 [00:06<00:10,  3.34s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 8798.62it/s]

 60%|██████    | 3/5 [00:09<00:06,  3.32s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 12698.47it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.36s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 9799.78it/s]

 78%|███████▊  | 7/9 [04:03<01:01, 30.50s/it]

window_size: 1750
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  1.0198704758659005
MakeWACAXExpDicUnknown Time:  3.883397586643696
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 11265.93it/s]

 20%|██        | 1/5 [00:03<00:13,  3.46s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 9560.76it/s]

 40%|████      | 2/5 [00:06<00:10,  3.39s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 7427.49it/s]

 60%|██████    | 3/5 [00:10<00:06,  3.36s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 7481.81it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.34s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 8530.21it/s]

 89%|████████▉ | 8/9 [04:25<00:27, 27.93s/it]

window_size: 2000
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  1.0818920619785786
MakeWACAXExpDicUnknown Time:  3.478485031053424
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 9993.58it/s]

 20%|██        | 1/5 [00:03<00:12,  3.25s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 9906.24it/s]

 40%|████      | 2/5 [00:06<00:09,  3.27s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 7496.52it/s]

 60%|██████    | 3/5 [00:09<00:06,  3.27s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8617.84it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.28s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 10631.95it/s]

100%|██████████| 9/9 [04:47<00:00, 31.90s/it]


In [ ]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "SMA"



preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    





EER_df_train_dict={}
EER_df_test_dict={}
for clf_type in tqdm(RIVAL_CLASSIFIER_TYPE_LST):

    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_type}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.winsize=rival_test_hyperparameters_df["SMA_winsize"][0]
    print(f"SMA winsize: {P.winsize}")

    P.SMA_per_win_winsize=P.winsize


    SMAed_dfList_exp2 = get_SMAed_dfList(raw_dfList_exp2, winsize=P.winsize)
    
    SMAed_dfList_exp2_user_47 = get_SMAed_dfList(raw_dfList_exp2_user_47, winsize=P.winsize)

    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": SMAed_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[P.winsize] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                       extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                       param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\SMA winsize: {P.winsize}\n")
        f.write(EER_df_train_dict[P.winsize].to_string())



        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["SMA_winsize"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

# 5. Butterworth frequency Cut-off + SMA winsize
## 5.1 Naive Approach
### Optimizing and Testin

In [22]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_5)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butter+SMA"


preprocessing_method="Naive"
time_of_execution = time.strftime("%Y-%m-%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/clip={P.scaler_clip}-Smoothing={P.smoothing}-Prep={preprocessing_method}-EER_df_test_dict.txt"
old_test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/old_{time_of_execution}_{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"


EER_df_test_dict={}

    
old_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
old_test_hyperparameters_df = pd.read_json(old_test_hyperparameters_file_name)
old_test_hyperparameters_df.to_json(f'{old_test_file_name[:-4]}_raw_df.json')

P.winsize=old_test_hyperparameters_df["SMA_winsize"][0]
P.cut_off_freq=old_test_hyperparameters_df["cut_off_freq"][0]

min_key= P.cut_off_freq, P.winsize
print(f"cut_off_freq: {P.cut_off_freq}, winsize: {P.winsize}")


ffted_dfList_exp1 = get_ffted_dfList(raw_dfList_exp1, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

SMAed_dfList_exp1 = get_SMAed_dfList(ffted_dfList_exp1, winsize=P.winsize)
SMAed_dfList_exp2 = get_SMAed_dfList(ffted_dfList_exp2, winsize=P.winsize)

ffted_dfList_exp1_user_47 = get_ffted_dfList(raw_dfList_exp1_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

SMAed_dfList_exp1_user_47 = get_SMAed_dfList(ffted_dfList_exp1_user_47, winsize=P.winsize)
SMAed_dfList_exp2_user_47 = get_SMAed_dfList(ffted_dfList_exp2_user_47, winsize=P.winsize)

dfList_dict={
            "dfList_exp1": SMAed_dfList_exp1,
            "dfList_exp2": SMAed_dfList_exp2,
            "dfList_exp1_user_47": SMAed_dfList_exp1_user_47,
            "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
}
    

EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=old_test_hyperparameters_df)
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq", "SMA_winsize"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

reseting experiment params successful!
train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
cut_off_freq: 43, winsize: 34


  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  5.661112245172262
MakeWACAXExpDicUnknown Time:  31.100371145643294
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 10140.97it/s]

 20%|██        | 1/5 [00:03<00:12,  3.08s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 8935.46it/s]

 40%|████      | 2/5 [00:06<00:09,  3.03s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 8090.86it/s]

 60%|██████    | 3/5 [00:08<00:05,  2.91s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8243.52it/s]

 80%|████████  | 4/5 [00:11<00:02,  2.91s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 12021.51it/s]

 11%|█         | 1/9 [00:52<07:02, 52.79s/it]

window_size: 250
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  2.9526474196463823
MakeWACAXExpDicUnknown Time:  16.301567037589848
Done extracting features


...


  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 9018.07it/s]

 20%|██        | 1/5 [00:03<00:14,  3.56s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 9108.15it/s]

 40%|████      | 2/5 [00:06<00:10,  3.48s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 7381.74it/s]

 60%|██████    | 3/5 [00:10<00:06,  3.44s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 11063.85it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.41s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 7577.79it/s]

 78%|███████▊  | 7/9 [03:34<00:51, 25.82s/it]

window_size: 1750
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  0.9936099136248231
MakeWACAXExpDicUnknown Time:  3.325911679305136
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 11422.40it/s]

 20%|██        | 1/5 [00:03<00:13,  3.37s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 9246.70it/s]

 40%|████      | 2/5 [00:06<00:09,  3.33s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 8076.84it/s]

 60%|██████    | 3/5 [00:09<00:06,  3.30s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8131.65it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.29s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 11360.52it/s]

 89%|████████▉ | 8/9 [03:55<00:24, 24.33s/it]

window_size: 2000
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  1.0469095390290022
MakeWACAXExpDicUnknown Time:  3.0555527517572045
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 9571.67it/s]

 20%|██        | 1/5 [00:03<00:13,  3.32s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 8655.19it/s]

 40%|████      | 2/5 [00:06<00:09,  3.31s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 6948.81it/s]

 60%|██████    | 3/5 [00:09<00:06,  3.29s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8056.67it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.26s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 12698.47it/s]

100%|██████████| 9/9 [04:15<00:00, 28.43s/it]


In [ ]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_5)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butter+SMA"



preprocessing_method="Naive"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    
indices = list(range(1, 50))
mesh = np.array(np.meshgrid(indices, indices))
index_pairs = mesh.T.reshape(-1, 2)

print(f"total cut_off_winsize_pairs: {index_pairs.shape}, choice_num: {CHOICE_NUM_PAIRS}")
cut_off_winsize_pairs = index_pairs[np.random.choice(index_pairs.shape[0], size=CHOICE_NUM_PAIRS, replace=False), :]








EER_df_train_dict={}
EER_df_test_dict={}

for clf_type in tqdm(RIVAL_CLASSIFIER_TYPE_LST):
    
    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_type}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.winsize=rival_test_hyperparameters_df["SMA_winsize"][0]
    P.cut_off_freq=rival_test_hyperparameters_df["cut_off_freq"][0]
    
    key_pair= P.cut_off_freq, P.winsize, clf_type
    print(f"cut_off_freq: {P.cut_off_freq}, winsize: {P.winsize}")

    
    ffted_dfList_exp1 = get_ffted_dfList(raw_dfList_exp1, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    SMAed_dfList_exp1 = get_SMAed_dfList(ffted_dfList_exp1, winsize=P.winsize)
    SMAed_dfList_exp2 = get_SMAed_dfList(ffted_dfList_exp2, winsize=P.winsize)
    
    ffted_dfList_exp1_user_47 = get_ffted_dfList(raw_dfList_exp1_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    SMAed_dfList_exp1_user_47 = get_SMAed_dfList(ffted_dfList_exp1_user_47, winsize=P.winsize)
    SMAed_dfList_exp2_user_47 = get_SMAed_dfList(ffted_dfList_exp2_user_47, winsize=P.winsize)
    
    dfList_dict={
                "dfList_exp1": SMAed_dfList_exp1,
                "dfList_exp2": SMAed_dfList_exp2,
                "dfList_exp1_user_47": SMAed_dfList_exp1_user_47,
                "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[key_pair] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                      extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                      param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\cut_off_freq, winsize: {key_pair}\n")
        f.write(EER_df_train_dict[key_pair].to_string())



        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key[:-1]] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key[:-1]].to_string())
#-------
#-------
key_column= ["cut_off_freq", "SMA_winsize"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

In [16]:
min_key

(43, 34, 'kNN')

## 5.2 Realworld Approach (per unknown window application of filter)
### Optimizing and Testing

In [23]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_5)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butter+SMA"


preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y-%m-%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/clip={P.scaler_clip}-Smoothing={P.smoothing}-Prep={preprocessing_method}-EER_df_test_dict.txt"
old_test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/old_{time_of_execution}_{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"


P.Butter_per_win_argdict={
    "filter_order": P.filter_order,
    "cut_off_freq": None,
    "sampling_freq": P.sampling_freq,
    "filtfilt": P.filtfilt,
    }


EER_df_test_dict={}

    
old_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
old_test_hyperparameters_df = pd.read_json(old_test_hyperparameters_file_name)
old_test_hyperparameters_df.to_json(f'{old_test_file_name[:-4]}_raw_df.json')

P.winsize=old_test_hyperparameters_df["SMA_winsize"][0]
P.cut_off_freq=old_test_hyperparameters_df["cut_off_freq"][0]

P.Butter_per_win_argdict["cut_off_freq"]=P.cut_off_freq
P.SMA_per_win_winsize=P.winsize

min_key= P.cut_off_freq, P.winsize
print(f"cut_off_freq: {P.cut_off_freq}, winsize: {P.winsize}")




ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
SMAed_dfList_exp2 = get_SMAed_dfList(ffted_dfList_exp2, winsize=P.winsize)

ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
SMAed_dfList_exp2_user_47 = get_SMAed_dfList(ffted_dfList_exp2_user_47, winsize=P.winsize)

dfList_dict={
            "dfList_exp1": raw_dfList_exp1,
            "dfList_exp2": SMAed_dfList_exp2,
            "dfList_exp1_user_47": raw_dfList_exp1_user_47,
            "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
}
    
        

EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=old_test_hyperparameters_df)
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq", "SMA_winsize"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

reseting experiment params successful!
train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
cut_off_freq: 26, winsize: 2


  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  5.4257441302761436
MakeWACAXExpDicUnknown Time:  35.97809991519898
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 10318.09it/s]

 20%|██        | 1/5 [00:03<00:12,  3.03s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 15551.74it/s]

 40%|████      | 2/5 [00:06<00:09,  3.01s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 8152.19it/s]

 60%|██████    | 3/5 [00:08<00:05,  2.96s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 10187.77it/s]

 80%|████████  | 4/5 [00:11<00:02,  2.95s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 6068.15it/s]

 11%|█         | 1/9 [01:39<13:17, 99.70s/it]

window_size: 250
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  3.0793705731630325
MakeWACAXExpDicUnknown Time:  18.598216144368052
Done extracting features


...


  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 9153.87it/s]

 20%|██        | 1/5 [00:03<00:13,  3.26s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 5968.84it/s]

 40%|████      | 2/5 [00:06<00:10,  3.37s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 6223.00it/s]

 60%|██████    | 3/5 [00:10<00:06,  3.34s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 9808.94it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.35s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 5416.20it/s]

 78%|███████▊  | 7/9 [05:09<01:06, 33.28s/it]

window_size: 1750
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  0.9596736952662468
MakeWACAXExpDicUnknown Time:  3.7512324703857303
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 12289.20it/s]

 20%|██        | 1/5 [00:03<00:14,  3.61s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 9556.40it/s]

 40%|████      | 2/5 [00:06<00:10,  3.47s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 6558.72it/s]

 60%|██████    | 3/5 [00:10<00:06,  3.37s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 14295.51it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.33s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 4642.80it/s]

 89%|████████▉ | 8/9 [05:34<00:30, 30.60s/it]

window_size: 2000
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  1.092708638869226
MakeWACAXExpDicUnknown Time:  3.2893404327332973
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 8526.74it/s]

 20%|██        | 1/5 [00:03<00:13,  3.38s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 6451.78it/s]

 40%|████      | 2/5 [00:06<00:10,  3.34s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 8182.41it/s]

 60%|██████    | 3/5 [00:09<00:06,  3.27s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8263.01it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.31s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 7841.29it/s]

100%|██████████| 9/9 [05:58<00:00, 39.84s/it]


In [ ]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_5)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butter+SMA"



preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    
indices = list(range(1, 50))
mesh = np.array(np.meshgrid(indices, indices))
index_pairs = mesh.T.reshape(-1, 2)

print(f"total cut_off_winsize_pairs: {index_pairs.shape}, choice_num: {CHOICE_NUM_PAIRS}")
cut_off_winsize_pairs = index_pairs[np.random.choice(index_pairs.shape[0], size=CHOICE_NUM_PAIRS, replace=False), :]







P.Butter_per_win_argdict={
    "filter_order": P.filter_order,
    "cut_off_freq": None,
    "sampling_freq": P.sampling_freq,
    "filtfilt": P.filtfilt,
    }


EER_df_train_dict={}
EER_df_test_dict={}

for clf_type in tqdm(RIVAL_CLASSIFIER_TYPE_LST):
    
    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_type}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.winsize=rival_test_hyperparameters_df["SMA_winsize"][0]
    P.cut_off_freq=rival_test_hyperparameters_df["cut_off_freq"][0]
    
    P.Butter_per_win_argdict["cut_off_freq"]=P.cut_off_freq
    P.SMA_per_win_winsize=P.winsize
    
    key_pair= P.cut_off_freq, P.winsize, clf_type
    print(f"cut_off_freq: {P.cut_off_freq}, winsize: {P.winsize}")
    


    
    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    SMAed_dfList_exp2 = get_SMAed_dfList(ffted_dfList_exp2, winsize=P.winsize)
    
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    SMAed_dfList_exp2_user_47 = get_SMAed_dfList(ffted_dfList_exp2_user_47, winsize=P.winsize)
    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": SMAed_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[key_pair] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                      extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                      param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\cut_off_freq, winsize: {key_pair}\n")
        f.write(EER_df_train_dict[key_pair].to_string())



        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]

EER_df_test_dict[min_key[:-1]] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key[:-1]].to_string())
#-------
#-------
key_column= ["cut_off_freq", "SMA_winsize"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

reseting experiment params successful!
train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
total cut_off_winsize_pairs: (2401, 2), choice_num: 13


  0%|          | 0/3 [00:00<?, ?it/s]

cut_off_freq: 26, winsize: 2



  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 337
len_exp2_user_47: 289
MakeWACAXExpDicOwner Time:  14.179950281977654
MakeWACAXExpDicUnknown Time:  85.55156582919881
Done extracting features


...



  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 9637.65it/s]


 33%|███▎      | 1/3 [06:23<12:46, 383.21s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 12943.39it/s]


 67%|██████▋   | 2/3 [12:29<06:13, 373.43s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 13831.18it/s]


100%|██████████| 3/3 [18:59<00:00, 379.69s/it]


  0%|          | 0/5 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 12606.87it/s]


 20%|██        | 1/5 [00:06<00:25,  6.33s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 17396.53it/s]


 40%|████      | 2/5 [00:12<00:18,  6.16s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 21913.81it/s]


 60%|██████    | 3/5 [00:18<00:12,  6.11s/it]


run: 3: 100%|██████████| 20/20 [00:00<00:00, 20228.14it/s]


 80%|████████  | 4/5 [00:24<00:06,  6.11s/it]


run: 4: 100%|██████████| 20/20 [00:00<00:00, 20997.77it/s]


100%|██████████| 5/5 [00:30<00:00,  6.09s/it]

 22%|██▏       | 2/9 [47:48<2:43:52, 1404.70s

window_size: 500
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 83
len_exp2_user_47: 71
MakeWACAXExpDicOwner Time:  5.226970039773732
MakeWACAXExpDicUnknown Time:  24.367419144138694
Done extracting features




  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 20257.45it/s]


 33%|███▎      | 1/3 [05:41<11:23, 341.58s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 10505.46it/s]


 67%|██████▋   | 2/3 [11:09<05:33, 333.54s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 7046.88it/s]


100%|██████████| 3/3 [16:57<00:00, 339.24s/it]


  0%|          | 0/5 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 17829.13it/s]


 20%|██        | 1/5 [00:04<00:18,  4.70s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 22429.43it/s]


 40%|████      | 2/5 [00:08<00:13,  4.44s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 20992.51it/s]


 60%|██████    | 3/5 [00:13<00:08,  4.38s/it]


run: 3: 100%|██████████| 20/20 [00:00<00:00, 9937.93it/s]


 80%|████████  | 4/5 [00:17<00:04,  4.35s/it]


run: 4: 100%|██████████| 20/20 [00:00<00:00, 16228.69it/s]


100%|██████████| 5/5 [00:21<00:00,  4.39s/it]

 33%|███▎      | 3/9 [1:06:00<2:06:09, 1261.65

window_size: 750
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 55
len_exp2_user_47: 47
MakeWACAXExpDicOwner Time:  4.457716403063387
MakeWACAXExpDicUnknown Time:  17.048368971794844
Done extracting features




  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 17600.94it/s]


In [ ]:
1

# 6. The effect of Varying Overlap

In [ ]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")



time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/overlap_Mean_EER_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/overlap_Mean_EER_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    




EER_df_train_dict={}
EER_df_test_dict={}
for overlap in tqdm(OVERLAP_EXP_RANGE):
    overlap*=0.01
    max_window_size=2000
    step_width = int(max_window_size * (1-overlap))
    max_num_windows=max(len(getIndices(sampleSize=max_window_size, step=step_width, numSamplePoints=P.num_sample_points_per_exp)), 100)
    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": raw_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": raw_dfList_exp2_user_47
    }
    

    EER_df_train_dict[overlap] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                     extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                     param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\overlap: {overlap}\n")
        f.write(EER_df_train_dict[overlap].to_string())



        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["overlap"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

# Summary

# 1.1 EER per window for each experiment

In [33]:
df=return_and_save_final_result_df_as_json(final_exp_results_path=FINAL_EXP_RESULTS_PATH, exp_path_name=EXP_PATH_NAME, window_size_lst=WINDOW_SIZE_LST)
df

Window Size            \
   mean rank cut_off_freq EMA_span SMA_winsize   type         125       250   
0         11            -        -           -   None    0.279934  0.227239   
1          5           37        -           -  Naive    0.274102  0.229973   
2          4           49        -           -   Real    0.270745  0.231470   
3          2            -       37           -  Naive    0.240382  0.213985   
4          7            -        9           -   Real    0.285539  0.237598   
5          8            -        -          41  Naive    0.240214  0.226334   
6         10            -        -           1   Real    0.279792  0.234405   
7          1           41       48           -  Naive    0.228424  0.204684   
8          3           40        3           -   Real    0.273475  0.226287   
9          9           43        -          34  Naive    0.265892  0.224624   
10         6           26        -           2   Real    0.276594  0.240286   

                                                                          \
         500       750      1000      1250      1500      1750      2000   
0   0.209224  0.189051  0.201003  0.196250  0.185309  0.192560  0.185667   
1   0.196787  0.193556  0.189729  0.197083  0.167366  0.167343  0.163778   
2   0.205408  0.185273  0.181599  0.184132  0.169300  0.178937  0.148944   
3   0.184190  0.185859  0.180542  0.187257  0.187284  0.186763  0.182500   
4   0.209331  0.191030  0.180949  0.185382  0.184033  0.164251  0.180833   
5   0.195448  0.191636  0.186341  0.191458  0.191852  0.207536  0.192000   
6   0.209277  0.190848  0.201111  0.175833  0.185267  0.192560  0.185722   
7   0.186653  0.185091  0.183523  0.190660  0.184280  0.192899  0.192167   
8   0.196078  0.191879  0.179892  0.179792  0.176790  0.159275  0.169611   
9   0.189705  0.189434  0.185474  0.190903  0.189218  0.190145  0.198667   
10  0.210977  0.187838  0.182114  0.183472  0.174074  0.188696  0.172444   

                        
        mean  variance  
0   0.207359  0.000918  
1   0.197746  0.001244  
2   0.195090  0.001326  
3   0.194307  0.000396  
4   0.202105  0.001417  
5   0.202536  0.000349  
6   0.206091  0.001056  
7   0.194264  0.000206  
8   0.194787  0.001232  
9   0.202674  0.000701  
10  0.201833  0.001235

# 1.2 Relative improvment of EER per window for each experiment

In [34]:
relative_df=return_and_save_final_relative_result_df_as_json(df, base_case_index=0, final_exp_results_path=FINAL_EXP_RESULTS_PATH, exp_path_name=EXP_PATH_NAME, window_size_lst=WINDOW_SIZE_LST)
relative_df

Window Size            \
   mean rank cut_off_freq EMA_span SMA_winsize   type         125       250   
0         11            -        -           -   None    0.000000  0.000000   
1          5           37        -           -  Naive    2.083530 -1.203373   
2          4           49        -           -   Real    3.282531 -1.862154   
3          2            -       37           -  Naive   14.128899  5.832406   
4          7            -        9           -   Real   -2.002261 -4.558763   
5          8            -        -          41  Naive   14.188966  0.398196   
6         10            -        -           1   Real    0.050645 -3.153364   
7          1           41       48           -  Naive   18.400782  9.925631   
8          3           40        3           -   Real    2.307312  0.418692   
9          9           43        -          34  Naive    5.016254  1.150670   
10         6           26        -           2   Real    1.193112 -5.741641   

                                                                               \
          500       750       1000       1250      1500       1750       2000   
0    0.000000  0.000000   0.000000   0.000000  0.000000   0.000000   0.000000   
1    5.944078 -2.382988   5.608737  -0.424628  9.682434  13.095835  11.789348   
2    1.823533  1.998290   9.653499   6.174805  8.638685   7.074762  19.778576   
3   11.964937  1.688395  10.179318   4.582449 -1.065956   3.010537   1.705566   
4   -0.051187 -1.047232   9.977080   5.537863  0.688430  14.701455   2.603232   
5    6.583915 -1.367814   7.294054   2.441614 -3.530979  -7.777220  -3.411131   
6   -0.025593 -0.951058  -0.053930  10.403397  0.022207   0.000000  -0.029922   
7   10.787638  2.094465   8.696238   2.848549  0.555185  -0.175615  -3.500898   
8    6.283192 -1.496046  10.502899   8.386412  4.596935  17.285499   8.647516   
9    9.328812 -0.203035   7.725495   2.724699 -2.109705   1.254390  -7.001795   
10  -0.838185  0.641163   9.397330   6.510970  6.062625   2.007025   7.121484   

                         
        mean   variance  
0   0.000000   0.000000  
1   4.635985 -35.589191  
2   5.917041 -44.505594  
3   6.294635  56.832921  
4   2.533941 -54.416506  
5   2.326307  61.989909  
6   0.611844 -15.084808  
7   6.315142  77.497026  
8   6.063359 -34.317690  
9   2.259783  23.572251  
10  2.665222 -34.595488

In [37]:
# round numbers
df.style.format(STYLER_ERR_FORMAT_DICT).hide(axis='index').to_latex()


'\\begin{tabular}{rllllrrrrrrrrrrr}\n\\multicolumn{5}{r}{} & \\multicolumn{9}{r}{Window Size} & \\multicolumn{2}{r}{} \\\\\nmean rank & cut_off_freq & EMA_span & SMA_winsize & type & 125 & 250 & 500 & 750 & 1000 & 1250 & 1500 & 1750 & 2000 & mean & variance \\\\\n11 & - & - & - & None & 0.2799 & 0.2272 & 0.2092 & 0.1891 & 0.2010 & 0.1963 & 0.1853 & 0.1926 & 0.1857 & 0.2074 & 0.0009 \\\\\n5 & 37 & - & - & Naive & 0.2741 & 0.2300 & 0.1968 & 0.1936 & 0.1897 & 0.1971 & 0.1674 & 0.1673 & 0.1638 & 0.1977 & 0.0012 \\\\\n4 & 49 & - & - & Real & 0.2707 & 0.2315 & 0.2054 & 0.1853 & 0.1816 & 0.1841 & 0.1693 & 0.1789 & 0.1489 & 0.1951 & 0.0013 \\\\\n2 & - & 37 & - & Naive & 0.2404 & 0.2140 & 0.1842 & 0.1859 & 0.1805 & 0.1873 & 0.1873 & 0.1868 & 0.1825 & 0.1943 & 0.0004 \\\\\n7 & - & 9 & - & Real & 0.2855 & 0.2376 & 0.2093 & 0.1910 & 0.1809 & 0.1854 & 0.1840 & 0.1643 & 0.1808 & 0.2021 & 0.0014 \\\\\n8 & - & - & 41 & Naive & 0.2402 & 0.2263 & 0.1954 & 0.1916 & 0.1863 & 0.1915 & 0.1919 & 0.2075 & 0.1

In [38]:
relative_df.style.format(STYLER_IMPROVEMENT_FORMAT_DICT).hide(axis='index').to_latex()


'\\begin{tabular}{rllllrrrrrrrrrrr}\n\\multicolumn{5}{r}{} & \\multicolumn{9}{r}{Window Size} & \\multicolumn{2}{r}{} \\\\\nmean rank & cut_off_freq & EMA_span & SMA_winsize & type & 125 & 250 & 500 & 750 & 1000 & 1250 & 1500 & 1750 & 2000 & mean & variance \\\\\n11 & - & - & - & None & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 \\\\\n5 & 37 & - & - & Naive & 2.08 & -1.20 & 5.94 & -2.38 & 5.61 & -0.42 & 9.68 & 13.10 & 11.79 & 4.64 & -35.59 \\\\\n4 & 49 & - & - & Real & 3.28 & -1.86 & 1.82 & 2.00 & 9.65 & 6.17 & 8.64 & 7.07 & 19.78 & 5.92 & -44.51 \\\\\n2 & - & 37 & - & Naive & 14.13 & 5.83 & 11.96 & 1.69 & 10.18 & 4.58 & -1.07 & 3.01 & 1.71 & 6.29 & 56.83 \\\\\n7 & - & 9 & - & Real & -2.00 & -4.56 & -0.05 & -1.05 & 9.98 & 5.54 & 0.69 & 14.70 & 2.60 & 2.53 & -54.42 \\\\\n8 & - & - & 41 & Naive & 14.19 & 0.40 & 6.58 & -1.37 & 7.29 & 2.44 & -3.53 & -7.78 & -3.41 & 2.33 & 61.99 \\\\\n10 & - & - & 1 & Real & 0.05 & -3.15 & -0.03 & -0.95 & -0.05 & 10.40 & 0.02

## 2.1 Gini coef

In [39]:
gini_result_df=return_and_save_final_Gini_df_as_json(final_exp_results_path=FINAL_EXP_RESULTS_PATH, exp_path_name=EXP_PATH_NAME, window_size_lst=WINDOW_SIZE_LST)
gini_result_df

Window Size            \
   mean rank cut_off_freq EMA_span SMA_winsize   type         125       250   
0          1            -        -           -   None    0.259356  0.324453   
1         11           37        -           -  Naive    0.268029  0.323810   
2          9           49        -           -   Real    0.270237  0.312839   
3          7            -       37           -  Naive    0.298167  0.309360   
4          4            -        9           -   Real    0.240130  0.296674   
5          5            -        -          41  Naive    0.309264  0.322940   
6          3            -        -           1   Real    0.259468  0.315541   
7          6           41       48           -  Naive    0.313314  0.335751   
8         10           40        3           -   Real    0.271727  0.319675   
9          2           43        -          34  Naive    0.282617  0.307426   
10         8           26        -           2   Real    0.256153  0.312217   

                                                                          \
         500       750      1000      1250      1500      1750      2000   
0   0.352182  0.397101  0.382142  0.417382  0.422679  0.395735  0.443128   
1   0.375698  0.393688  0.445884  0.438251  0.494869  0.519938  0.515679   
2   0.378444  0.422112  0.428621  0.421185  0.461454  0.450546  0.524982   
3   0.374225  0.379676  0.430968  0.414585  0.415437  0.404920  0.468307   
4   0.316293  0.410627  0.428993  0.427914  0.421407  0.456882  0.427455   
5   0.373247  0.388238  0.400003  0.404566  0.401649  0.403802  0.442316   
6   0.352753  0.393236  0.381726  0.459742  0.422662  0.395735  0.443165   
7   0.375023  0.394174  0.405198  0.409322  0.418416  0.398884  0.407025   
8   0.362018  0.381814  0.433350  0.433106  0.455266  0.562980  0.490909   
9   0.365868  0.377817  0.381362  0.413229  0.406317  0.445568  0.437304   
10  0.345853  0.410026  0.448803  0.423072  0.477121  0.449047  0.502133   

                        
        mean  variance  
0   0.377129  0.003261  
1   0.419538  0.007594  
2   0.407824  0.006007  
3   0.388405  0.003069  
4   0.380709  0.005759  
5   0.382892  0.001777  
6   0.380448  0.004002  
7   0.384123  0.001317  
8   0.412316  0.007925  
9   0.379723  0.003042  
10  0.402714  0.006633

In [40]:
return_and_save_final_relative_gini_result_df_as_json(gini_result_df, base_case_index=0, final_exp_results_path=FINAL_EXP_RESULTS_PATH, 
                                                      exp_path_name=EXP_PATH_NAME, window_size_lst=WINDOW_SIZE_LST)

Window Size            \
   mean rank cut_off_freq EMA_span SMA_winsize   type         125       250   
0          1            -        -           -   None    0.000000  0.000000   
1         11           37        -           -  Naive   -3.344071  0.198204   
2          9           49        -           -   Real   -4.195334  3.579669   
3          7            -       37           -  Naive  -14.964151  4.651838   
4          4            -        9           -   Real    7.413027  8.561903   
5          5            -        -          41  Naive  -19.242888  0.466369   
6          3            -        -           1   Real   -0.043149  2.746792   
7          6           41       48           -  Naive  -20.804604 -3.482082   
8         10           40        3           -   Real   -4.769679  1.472820   
9          2           43        -          34  Naive   -8.968711  5.247978   
10         8           26        -           2   Real    1.234993  3.771312   

                                                                     \
          500       750       1000       1250       1500       1750   
0    0.000000  0.000000   0.000000   0.000000   0.000000   0.000000   
1   -6.677072  0.859425 -16.680280  -4.999849 -17.079300 -31.385477   
2   -7.456874 -6.298519 -12.162849  -0.911114  -9.173730 -13.850398   
3   -6.259021  4.387854 -12.777126   0.670248   1.713197  -2.321077   
4   10.190445 -3.406311 -12.260315  -2.523310   0.300804 -15.451570   
5   -5.981103  2.231899  -4.674080   3.070588   4.975278  -2.038463   
6   -0.162099  0.973197   0.108754 -10.148904   0.004063   0.000000   
7   -6.485482  0.736971  -6.033430   1.931064   1.008508  -0.795756   
8   -2.792819  3.849606 -13.400384  -3.767202  -7.709679 -42.261794   
9   -3.886137  4.856110   0.203904   0.995057   3.871014 -12.592491   
10   1.797116 -3.254977 -17.444130  -1.363140 -12.880323 -13.471633   

                                      
         2000       mean    variance  
0    0.000000   0.000000    0.000000  
1  -16.372546 -11.245460 -132.865286  
2  -18.471986  -8.139359  -84.211021  
3   -5.682164  -2.990104    5.885598  
4    3.536828  -0.949243  -76.599719  
5    0.183190  -1.528131   45.494286  
6   -0.008339  -0.880044  -22.727049  
7    8.147246  -1.854655   59.617929  
8  -10.782642  -9.330319 -143.032317  
9    1.314207  -0.687984    6.719072  
10 -13.315701  -6.784244 -103.392845